In [ ]:
from datetime import datetime, timezone
import glob

from climatology import compute_clim_mon
from xr_wrap import open_mfdataset_wrap

# Set options for climatology generation, determine list of input files

In [ ]:
base_dir = '/glade/collections/cdg/data/CMIP6/CMIP'
institute = 'NCAR'
model = 'CESM2'
experiment = 'historical'
rip = 'r7i1p1f1'
MIP_table = 'Omon'
varname = 'fgco2'
grid = 'gn'

yr_lo = '1999'
yr_hi = '2000'

dir_in = '/'.join([base_dir, institute, model, experiment, rip, MIP_table, varname, grid, 'latest'])
fname_pattern = '_'.join([varname, MIP_table, model, experiment, rip, grid, '*.nc'])
fnames = glob.glob(dir_in+'/'+fname_pattern)
fnames.sort()
print(fnames)

# Generate climatology from list of input files

In [ ]:
time_name = 'time'

# data_vars = 'minimal', to avoid introducing time dimension
# to time-invariant fields when there are multiple files
ds_in = open_mfdataset_wrap(fnames, chunks={time_name: 12}, data_vars='minimal', combine='by_coords')

ds_clim = compute_clim_mon(ds_in, yr_lo, yr_hi)

# change some global attributes that are not appropriate after postprocessing, document input files
ds_clim.attrs['contact'] = 'klindsay@ucar.edu'
ds_clim.attrs['creation_date'] = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z")
ds_clim.attrs['input_file_list'] = ' '.join(fnames)

# Write generated climatology to a file

In [ ]:
dir_out = '/glade/scratch/klindsay/CESM2_CMIP6_ocnBgchem_tst'

yr_fname_string = yr_lo + '-' + yr_hi

fname_out = '_'.join([varname, MIP_table, model, experiment, rip, grid, yr_fname_string, 'mon_clim.nc'])
print(fname_out)

fname_out_full = '/'.join([dir_out, fname_out])
ds_clim.to_netcdf(fname_out_full)

# Correctness of climatology is checked with ferret script mon_clim_test_verify.jnl